In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Adds 8 hours to timestamp (for GMT-8 tz) to make GMT time. Replace with your offset from GMT.
TZ_ADJ = 8

data_path = './data.csv'

In [2]:
def rsi(df, periods=14):
    """
    Returns a pd.Series with the relative strength index.
    From https://www.roelpeters.be/many-ways-to-calculate-the-rsi-in-python-pandas/
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    # Use exponential moving average
    ma_up = up.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [30]:
data = pd.read_csv(data_path)

# adjusting from GMT-8 local tz
data['time'] = data['time'].apply(lambda x: datetime.fromtimestamp(x) + timedelta(seconds=60*60*TZ_ADJ))

data['rsi'] = rsi(data)

In [31]:
data

,time,open,high,low,close,rsi
0,2020-07-21 20:00:00,1.26675,1.26675,1.26498,1.26551,NaN
1,2020-07-21 21:00:00,1.26551,1.26795,1.26546,1.26791,NaN
2,2020-07-21 22:00:00,1.26791,1.27009,1.26787,1.26926,NaN
3,2020-07-21 23:00:00,1.26926,1.27162,1.26723,1.26741,NaN
4,2020-07-22 00:00:00,1.26741,1.26962,1.26712,1.26953,NaN
...,...,...,...,...,...,...
9029,2022-01-01 08:00:00,1.35449,1.35464,1.35303,1.35431,65.016548
9030,2022-01-01 09:00:00,1.35431,1.35462,1.35307,1.35345,61.113064
9031,2022-01-01 10:00:00,1.35345,1.35435,1.35293,1.35414,63.030863
9032,2022-01-01 11:00:00,1.35414,1.35416,1.35276,1.35328,59.117506


In [38]:
# Find where RSI goes below 30 or above 70, and then retracts on the next bar. 
# Using the close of that bar as the entry, what is the MAE and MFE of the next 4 bars? 

In [32]:
# If RSI is below 30, signal = 1; if RSI is above 70, signal = -1
data['long_signal'] = data['rsi'].apply(lambda x : 1 if x < 30 else 0)
data['short_signal'] = data['rsi'].apply(lambda x : -1 if x > 70 else 0)

In [33]:
data['lowest_low'] = data['low'].rolling(4).min()
data['highest_high'] = data['high'].rolling(4).max()

In [34]:
# calculate MAE
data['long_mae'] = ((data['close'].shift(4) - data['lowest_low']) * data['long_signal'].shift(5)).clip(lower=0)
data['short_mae'] = ((data['close'].shift(4) - data['highest_high']) * data['short_signal'].shift(5)).clip(lower=0)

In [37]:
# calculate mfe
data['long_mfe'] = ((data['highest_high'] - data['close'].shift(4)) * data['long_signal'].shift(5)).clip(lower=0) 
data['short_mfe'] = ((data['lowest_low'] - data['close'].shift(4)) * data['short_signal'].shift(5)).clip(lower=0) 